## Twilio sms Sample Code

* Sign up for a trial account at

>```https://www.twilio.com/try-twilio```

* Please install the twilio package in your conda environment by running the following command in the terminal

>``` pip install twilio ```

* Add the twilio API key parameters to your .env file

>```TWILIO_ACCOUNT_SID = "your-account-sid-goes-here"```
>```TWILIO_AUTH_TOKEN = "your-auth-token-goes-here"```

**REF: https://www.twilio.com/docs/libraries/python**
https://www.twilio.com/docs/sms/tutorials/how-to-send-sms-messages-python?code-sample=code-send-an-mms-message-with-an-image-url&code-language=Python&code-sdk-version=6.x

In [34]:
import os
from dotenv import load_dotenv
from twilio.rest import Client
import psycopg2
from datetime import date, timedelta
import numpy as np

In [35]:
# Load .env environment variables
# Note: Replace below file name with your .env file name
load_dotenv("YH.env")

True

In [36]:
# Read the API keys from the .env file
TWILIO_ACCOUNT_SID = os.getenv("TWILIO_ACCOUNT_SID") 
TWILIO_AUTH_TOKEN = os.getenv("TWILIO_AUTH_TOKEN")

# Create the api object
twilio_api = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)

# Read connection string from the .env file 
conn_string = os.getenv("DB_STR") 

# This is a function that 
# calls the message stream object in twilio
def fetch_sms():
    return twilio_api.messages.stream()

In [37]:
# run the function above and capture the returned object
sms = fetch_sms()
sms

<generator object Version.stream at 0x7fc41010b350>

In [38]:
# go through the list of messages 
for sms in twilio_api.messages.list():
    print(f"{sms.status} on {sms.date_sent} | direction {sms.direction} - message: {sms.body} to phone number {sms.from_}")

received on 2021-05-08 16:26:53+00:00 | direction inbound - message: ADD 
Micheal to phone number +19176015305
received on 2021-05-08 15:59:13+00:00 | direction inbound - message: START to phone number +19175437108
received on 2021-05-08 15:58:02+00:00 | direction inbound - message: STOP to phone number +19175437108
received on 2021-05-08 15:55:01+00:00 | direction inbound - message: TICKERS TSLA, HD, GE to phone number +19175437108
received on 2021-05-08 15:53:12+00:00 | direction inbound - message: TICKERS TSLA to phone number +19176015305
received on 2021-05-08 15:53:00+00:00 | direction inbound - message: TICKER TSLA to phone number +19176015305
received on 2021-05-08 15:52:27+00:00 | direction inbound - message: Tickers TSLA to phone number +19176015305
received on 2021-05-08 15:51:43+00:00 | direction inbound - message: ADD Farah  to phone number +19176015305
received on 2021-05-08 15:51:39+00:00 | direction inbound - message: TSLA, HD, GE to phone number +19175437108
received on

### Assumptions/Notes

In [39]:
# up to 10 tickers 
# command format 
# ADD Name
# TICKERS FB, ADM, AMD

### Functions

In [40]:
# Parse sms function
def parse_message(message_body):
    """Parses the given string by spliting it into 3 parts

    Parameters
    ----------
    message_body : str
        The message to be parsed

    Returns
    ----------
    str: Returns 2 strings.

   """

    message_split = message_body.split(' ', 1)
    
    if (len(message_split)<2) or message_split[1].replace(' ','')=='':
        message_split[0] = 'INVALID'
        message_split.append('')
    
    command = message_split[0].replace(' ', '')
    content = message_split[1].replace(' ', '')

    return command.strip(), content.strip()

In [41]:
def create_db_connection(conn_string):
    """Creates a connection to database

    Parameters
    ----------
    conn_string : str
        This is the connection string.

    Returns
    ----------
    obj: Returns a psycopg2.extensions.connection object.
    
    """
    conn = None
    try:
        conn = psycopg2.connect(conn_string)
        # for debug - to be removed for production
        print("MySQL Database connection successful")
    except Exception as err:
        print(f"Error: '{err}'")

    return conn

In [42]:
def execute_query(connection, query):
    """Executes a query to the databse.

    Parameters
    ----------
    conn : object 
        This is a psycopg2.extensions.connection object.
    query : str
        This is a SQL query string. 

    Returns
    ----------
    None: This function does not return any value.
    
    """
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        # for debug - to be removed for production
        print("Query successful")
    except Exception as err:
        print(f"Error: '{err}'")

In [43]:
def add_user(conn, name, number):
    """Adds a user to the databse 

    Parameters
    ----------
    conn : object 
        This is a psycopg2.extensions.connection object.
    name : str
        This is the name of the person.
    number : str
        This is the phone number of the person. 

    Returns
    ----------
    None: This function does not return any value.
    
    """
    #execute the sql 
    query = f"insert into client (user_name, phone_number) Select '{name}', '{number}' Where not exists(select * from client where phone_number='{number}')"
    execute_query(conn, query)
    conn.close()

In [44]:
def add_user_portfolio(conn, content, number):
    """Adds a portfolio to the database

    Parameters
    ----------
    conn : object 
        This is a psycopg2.extensions.connection object.
    content : str
        A comma separated string of ticker symbols.
    number : str
        This is the phone number of the person. 

    Returns
    ----------
    None: This function does not return any value.
    
    """
    query = f"insert into user_portfolio (user_id, tickers) values ((select user_id from client where phone_number='{number}'), '{content}') "
    execute_query(conn, query)
    conn.close()

In [45]:
def content_to_list(content):
    """Converts comma separated string to a list

    Parameters
    ----------
    content : str
        A comma separated string of ticker symbols.

    Returns
    ----------
    List: Returns the corresponding list item.
    
    """
    return content.replace(" ", "").split(',')

In [46]:
def execute_command(command, phone, content):
    """This functions executes commands received from the sms engine. 

    Parameters
    ----------
    command : str 
        This is the command type to be executed ADD = add user, TICKERS = add profile.
    phone : str
        This is the phone number of the person. 
    content : str
        This is the content to be added to the database.

    Returns
    ----------
    None: This function does not return any value.
    
    """
    if command=='ADD':
        # for debug - to be removed for production
        print('Adding a user!')
        # connect to db
        conn = create_db_connection(conn_string)
        # add user to db
        add_user(conn, content, phone)
    elif command=='TICKERS':
        # for debug - to be removed for production
        print('Adding portfolio')
        # add portfolio to db
        conn = create_db_connection(conn_string)
        add_user_portfolio(conn, content, phone)
        ticker_list = content_to_list(content)
        # run MC sim - RN
        # add to events - RN
        # Visualize the data - JF
            # plots to be made  - JF
        # message back results - YH
    else:  print('Invalid entry!')

### Main Program Area

In [47]:
# check inbound messages for today 

today = date.today()

for sms in twilio_api.messages.list():
    if (sms.direction == 'inbound') and (sms.date_created.date() == (today)):
        command, content = parse_message(sms.body)
        phone = sms.from_
        execute_command(command, phone, content)
        print(sms.body)
        print('---')

Adding a user!
MySQL Database connection successful
Query successful
ADD 
Micheal
---
Invalid entry!
START
---
Invalid entry!
STOP
---
Adding portfolio
MySQL Database connection successful
Query successful
TICKERS TSLA, HD, GE
---
Adding portfolio
MySQL Database connection successful
Query successful
TICKERS TSLA
---
Invalid entry!
TICKER TSLA
---
Invalid entry!
Tickers TSLA
---
Adding a user!
MySQL Database connection successful
Query successful
ADD Farah 
---
Invalid entry!
TSLA, HD, GE
---
Adding a user!
MySQL Database connection successful
Query successful
ADD 
Farah 
---
Adding a user!
MySQL Database connection successful
Query successful
ADD Rina
---
Invalid entry!
2 +19175437108 TSLA, HD, GE
---
Invalid entry!
2 +15185888001 FB, AMZN
---
Invalid entry!
1 +22222 19176015305
---
Invalid entry!
1 +15185888001 Justin
---
Invalid entry!
1 +19175437108 Rina
---
